In [1]:
!pip install transformers
import transformers
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case = False)

import tensorflow as tf
import numpy as np
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.5 MB/s eta 0:00:00


In [2]:
dataset = pd.read_csv("/content/drive/MyDrive/Youtube_Scraped/cleaned_data.csv", index_col=False)
dataset.head()

,Titles,Target
0,김뿡,0
1,꼽줘버린 김뿡,0
2,분명 지누형 생일기념 탈출맵이라고 했는데 김뿡,0
3,2023 08 25 아웃라스트 트라이얼 w 지누 코렛트 탬탬버린,0
4,같은 파티원 유니온 상태 크아아아악 김뿡,1


In [3]:
dataset = dataset.sample(frac = 1)
dataset.head()

,Titles,Target
264,탬탬버린 메이플 조별과제 절망편,0
460,북한에 비둘기가 없는 이유ㄷㄷ 비둘기의 소름 돋는 비밀 TOP5 빠퀴2tv,1
193,글로벌K 밤마다 육교로 모이는 중국 라이브 스트리머들 경쟁 고조 KBS 2023 0...,0
80,픽셀모임에서 패드립 켠왕하는 사람 김뿡,1
517,이세계아이돌 ISEGYE IDOL LOCKDOWN 락다운 메르헨 Marchen,1


In [4]:
ten_perc = int(len(dataset) * 0.1)
train_titles = np.array(dataset["Titles"][:-ten_perc], dtype = str)
train_labels = np.array(dataset["Target"][:-ten_perc])

valid_titles = np.array(dataset["Titles"][-ten_perc:], dtype = str)
valid_labels = np.array(dataset["Target"][-ten_perc:])

train_titles[:5], valid_titles[:5]

(array(['탬탬버린 메이플 조별과제 절망편', '북한에 비둘기가 없는 이유ㄷㄷ 비둘기의 소름 돋는 비밀 TOP5 빠퀴2tv',
        '글로벌K 밤마다 육교로 모이는 중국 라이브 스트리머들 경쟁 고조 KBS 2023 03 01',
        '픽셀모임에서 패드립 켠왕하는 사람 김뿡',
        '이세계아이돌 ISEGYE IDOL LOCKDOWN 락다운 메르헨 Marchen'], dtype='<U97'),
 array(['상황극', '유리온실 다시 돌아온 유리의 게임 방송', '해커들 진짜 미친거 아님',
        '2023 06 22 저스트채팅 Just Chatting',
        '20230818 KIDDING 키딩 음원 발매 담력시험 왁굳님 시점으로 다시보기 아이네 다시보기'],
       dtype='<U92'))

In [5]:
i = 35
tokenized_data = tokenizer.encode_plus(train_titles[i],
                                       add_special_tokens = True,
                                       max_length = 30,
                                       pad_to_max_length = True,
                                       truncation = True)

tokenized_data = dict(tokenized_data)
labels = np.array(train_labels[i])

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [6]:
def bert_encode(titles, maximum_length):
  input_ids = []
  attention_masks = []

  for items in titles:
    encoded = tokenizer.encode_plus(items,
                                    add_special_tokens = True,
                                    max_length = maximum_length,
                                    pad_to_max_length = True,
                                    truncation = True)
    input_ids.append(encoded['input_ids'])
    attention_masks.append(encoded['attention_mask'])
  return np.array(input_ids), np.array(attention_masks)

In [7]:
train_input_ids, train_attention_masks = bert_encode(train_titles, 30)
valid_input_ids, valid_attention_masks = bert_encode(valid_titles, 30)

In [8]:
from tensorflow.keras.optimizers import Adam

def create_model(bert_model):
  input_ids = tf.keras.Input(shape=(30,), dtype='int32')
  attention_masks = tf.keras.Input(shape=(30,), dtype='int32')

  output = bert_model([input_ids, attention_masks])
  output = output[1]
  output = tf.keras.layers.Dense(32, activation = 'relu')(output)
  output = tf.keras.layers.Dropout(0.2)(output)
  output = tf.keras.layers.Dense(1, activation = 'sigmoid')(output)

  model = tf.keras.models.Model(inputs = [input_ids, attention_masks], outputs = output)
  model.compile(Adam(learning_rate = 0.00001), loss = 'binary_crossentropy', metrics = ['accuracy'])

  return model

In [9]:
from transformers import TFBertModel
bert_model = TFBertModel.from_pretrained('bert-base-multilingual-cased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [10]:
model = create_model(bert_model)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 30)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 30)]         0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  177853440   ['input_1[0][0]',                
                                thPoolingAndCrossAt               'input_2[0][0]']                
                                tentions(last_hidde                                               
                                n_state=(None, 30,                                            

In [11]:
history = model.fit([train_input_ids, train_attention_masks], train_labels,
                    validation_split=0.2, epochs = 3, batch_size = 32)

Epoch 1/3
18/18 [==============================] - 302s 14s/step - loss: 0.6979 - accuracy: 0.5089 - val_loss: 0.6864 - val_accuracy: 0.5390
Epoch 2/3
18/18 [==============================] - 260s 14s/step - loss: 0.6887 - accuracy: 0.5567 - val_loss: 0.6843 - val_accuracy: 0.5816
Epoch 3/3
18/18 [==============================] - 284s 16s/step - loss: 0.6854 - accuracy: 0.5461 - val_loss: 0.6614 - val_accuracy: 0.5957


In [13]:
model.evaluate([valid_input_ids, valid_attention_masks], valid_labels)

3/3 [==============================] - 10s 3s/step - loss: 0.6856 - accuracy: 0.4615


[0.6856147646903992, 0.4615384638309479]

In [14]:
a = model.predict([valid_input_ids, valid_attention_masks])
a= tf.squeeze(a)

3/3 [==============================] - 12s 2s/step


In [15]:
model.save("bert_model_v1.h5")

In [16]:
from google.colab import files
files.download("bert_model_v1.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>